In [1]:
import pygame
import random
import math
import time
CAR_MOVEMENT_DELAY = 1000  # Delay in milliseconds
CAR_CHECK_DELAY = 1  # Adjust the delay time as needed (in seconds)
delivery_points = []

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def draw_grid_with_entities(screen, width, height, rows, cols, building_image, building_locations, robot_image, robot_location, police_image, police_location, car1_image, car1_image_locations, car2_image, car2_image_locations):
    """
    Draw a grid on the screen with building images, robot image, police image, car1 images, and car2 images at the specified locations.

    Parameters:
        screen (pygame.Surface): The surface to draw on.
        width (int): Width of the screen.
        height (int): Height of the screen.
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_image (pygame.Surface): The image of the building.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_image (pygame.Surface): The image of the robot.
        robot_location (tuple): The location of the robot (row, col).
        police_image (pygame.Surface): The image of the police.
        police_location (tuple): The location of the police (row, col).
        car1_image (pygame.Surface): The image of car1.
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image (pygame.Surface): The image of car2.
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
    """
    cell_width = width // cols
    cell_height = height // rows

    screen.fill((0, 0, 0))  # Set background color to black

    for x in range(0, width + 1, cell_width):  # Increment the range by 1 to ensure the rightmost line is drawn
        pygame.draw.line(screen, (255, 255, 255), (x, 0), (x, height))  # Set border color to white
    for y in range(0, height + 1, cell_height):  # Increment the range by 1 to ensure the bottom line is drawn
        pygame.draw.line(screen, (255, 255, 255), (0, y), (width, y))  # Set border color to white

    # Adjust cell size to increase the size of the cell in the grid
    cell_width += 2
    cell_height += 2

    # Draw buildings
    for location in building_locations:
        building_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        building_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        building_image_resized = pygame.transform.scale(building_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(building_image_resized, (building_x, building_y))

    # Draw robot
    robot_x = robot_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    robot_y = robot_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    robot_image_resized = pygame.transform.scale(robot_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
    screen.blit(robot_image_resized, (robot_x, robot_y))

    # Draw police
    police_x = police_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    police_y = police_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    police_image_resized = pygame.transform.scale(police_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness

    screen.blit(police_image_resized, (police_x, police_y))

    # Draw car1 images
    for location in car1_image_locations:
        car1_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car1_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car1_image_resized = pygame.transform.scale(car1_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car1_image_resized, (car1_x, car1_y))

    # Draw car2 images
    for location in car2_image_locations:
        car2_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car2_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car2_image_resized = pygame.transform.scale(car2_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car2_image_resized, (car2_x, car2_y))


In [3]:
def generate_location(rows, cols, excluded_locations):
    """
    Generate a random location that does not overlap with the excluded locations.

    Parameters:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        excluded_locations (list): List of locations to avoid [(row, col), ...].

    Returns:
        tuple: The random location (row, col).
    """
    while True:
        location = (random.randint(0, rows - 1), random.randint(0, cols - 1))
        if location not in excluded_locations and location != (0, 0):  # Exclude the (0, 0) location
            return location


In [4]:
def move_police(police_location, rows, cols, building_locations, robot_location, car1_image_locations, car2_image_locations, prev_direction=None):
    """
    Move the police randomly within the grid, ensuring it does not collide with buildings, robot, or other cars.

    Parameters:
        police_location (tuple): The current location of the police (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of the police (dx, dy).

    Returns:
        tuple: The new location of the police (row, col).
        tuple: The new movement direction of the police (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby cars or robot
    nearby_entities = []
    for dx, dy in movements:
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) in car1_image_locations or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if not (0 <= new_row < rows and 0 <= new_col < cols) or (new_row, new_col) in building_locations:
            continue

        # If the new location is valid, return it along with the movement direction
        return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return police_location, (0, 0)


def move_car1(car1_location, rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction=None):
    """
    Move car1 randomly within the grid, ensuring it does not collide with buildings, robot, police, or other cars.

    Parameters:
        car1_location (tuple): The current location of car1 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        police_location (tuple): The location of the police (row, col).
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car1 (dx, dy).

    Returns:
        tuple: The new location of car1 (row, col).
        tuple: The new movement direction of car1 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == police_location or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in building_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != police_location) and \
           ((new_row, new_col) not in car2_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car1_location, (0, 0)


def move_car2(car2_location, rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction=None):
    """
    Move car2 randomly within the grid, ensuring it does not collide with buildings, robot, police, or other cars.

    Parameters:
        car2_location (tuple): The current location of car2 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        police_location (tuple): The location of the police (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car2 (dx, dy).

    Returns:
        tuple: The new location of car2 (row, col).
        tuple: The new movement direction of car2 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided and it's in the list of movements, prioritize that direction
    if prev_direction and prev_direction in movements:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == police_location or \
           (new_row, new_col) in car1_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in building_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != police_location) and \
           ((new_row, new_col) not in car1_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car2_location, (0, 0)




In [5]:
def draw_item_buttons(screen, item_images):
    # Fixed size for the item buttons
    item_button_size = 50
    
    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300

    # Draw item buttons below the grid
    for i, item_image in enumerate(item_images):
        # Calculate the position of the item button
        button_rect = pygame.Rect(start_x + i * item_button_size, screen.get_height() - item_button_size, item_button_size, item_button_size)
        
        pygame.draw.rect(screen, (0, 255, 0), button_rect, 1)  # Draw button border

        # Resize the item image to fit the button size (square dimensions)
        resized_item_image = pygame.transform.scale(item_image, (item_button_size - 2, item_button_size - 2))  # Subtract 2 for border thickness

        # Calculate the position to center the image within the button
        image_x = start_x + i * item_button_size + (item_button_size - resized_item_image.get_width()) // 2
        image_y = screen.get_height() - item_button_size + (item_button_size - resized_item_image.get_height()) // 2

        screen.blit(resized_item_image, (image_x, image_y))  # Draw item image




def handle_item_click(index, item_images, screen, rows, cols):

    # Remove the clicked item image from the list
    del item_images[index]

    # Calculate the area of the screen to redraw (where the item buttons are located)
    cell_height = screen.get_height() // (rows + 1)

    # Calculate the width of each column (grid cells)
    cell_width = screen.get_width() // cols

    # Calculate the width and height of each item button (square dimensions)
    item_button_size=50

    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300

    # Redraw only the button area to reflect the changes
    screen.fill((255, 255, 255), (0, screen.get_height() - item_button_size, screen.get_width(), item_button_size))  # Fill only the button area with white color
    draw_item_buttons(screen, item_images)  # Draw item buttons
    pygame.display.update((0, screen.get_height() - item_button_size, screen.get_width(), item_button_size))  # Update only the button area of the display




In [6]:
def euclidean_distance(start, goal):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((start[0] - goal[0]) ** 2 + (start[1] - goal[1]) ** 2)

def heuristic_with_obstacles(start, goal, obstacles):
    """
    Calculate the heuristic function with obstacles using Euclidean distance.
    Penalize the heuristic if there are obstacles between the start and the goal.
    """
    distance = euclidean_distance(start, goal)
    # Check for obstacles between start and goal
    for obstacle in obstacles:
        if obstacle[0] > min(start[0], goal[0]) and obstacle[0] < max(start[0], goal[0]) and obstacle[1] > min(start[1], goal[1]) and obstacle[1] < max(start[1], goal[1]):
            # Penalize the heuristic if there is an obstacle between start and goal
            distance += 100  # Adjust the penalty as needed
            break
    return distance

def best_first_search(start, goal, obstacles):
    """
    Perform Best-First Search algorithm to find the optimal path from start to goal.
    """
    visited = set()  # Set to store visited nodes
    queue = [(heuristic_with_obstacles(start, goal, obstacles), [start])]  # Priority queue for nodes to be explored

    while queue:
        # Get the node with the lowest heuristic value
        queue.sort()  # Sort the queue based on heuristic value
        _, path = queue.pop(0)
        node = path[-1]

        # Check if the goal is reached
        if node == goal:
            return path

        # Explore neighboring nodes
        for neighbor in get_neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                new_path = list(path)
                new_path.append(neighbor)
                queue.append((heuristic_with_obstacles(neighbor, goal, obstacles), new_path))

    return None  # No path found

def get_neighbors(node):
    """
    Get neighboring nodes of a given node.
    For simplicity, assume 4-connected grid.
    """
    neighbors = []
    x, y = node
    # Add all possible neighboring nodes (up, down, left, right)
    neighbors.append((x - 1, y))
    neighbors.append((x + 1, y))
    neighbors.append((x, y - 1))
    neighbors.append((x, y + 1))
    return neighbors

In [7]:
def generate_delivery_points(rows, cols, excluded_locations):
    """
    Generate 5 random delivery points that do not overlap with excluded locations.

    Parameters:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        excluded_locations (list): List of locations to avoid [(row, col), ...].

    Returns:
        list: List of 5 random delivery points [(row, col), ...].
    """
    global delivery_points
    if not delivery_points:
        while len(delivery_points) < 5:
            location = generate_location(rows, cols, excluded_locations)
            delivery_points.append(location)
            excluded_locations.append(location)

    return delivery_points

def delivery_buttons(delivery_points, cell_width, cell_height):
    """
    Create buttons for the cells where delivery points are located.

    Parameters:
        delivery_points (list): List of delivery points [(row, col), ...].
        cell_width (int): Width of each cell in the grid.
        cell_height (int): Height of each cell in the grid.

    Returns:
        list: List of button rects for each delivery point.
    """
    button_rects = []
    for delivery_point in delivery_points:
        delivery_x = delivery_point[1] * cell_width
        delivery_y = delivery_point[0] * cell_height
        button_rect = pygame.Rect(delivery_x, delivery_y, cell_width, cell_height)
        button_rects.append(button_rect)
    return button_rects

def display_coordinates(x, y, screen, cell_width, cell_height, point_image_resized, delivery_points):
    print("Clicked on delivery point at (x={}, y={})".format(x, y))
    for i, point in enumerate(delivery_points):
        point_x = point[1] * cell_width
        point_y = point[0] * cell_height
        if point_x == x and point_y == y:
            pygame.draw.rect(screen, (255, 0, 0), (point_x, point_y, cell_width, cell_height))
            screen.blit(point_image_resized, (point_x, point_y))


In [8]:
def is_blocked_or_occupied(next_location, building_locations, car1_image_locations, car2_image_locations , police_location):
    x, y = next_location
    if (x, y) in car1_image_locations or (x, y) in car2_image_locations or (x,y) in police_location:
        return True  # Location is occupied by a car
    return False

In [9]:
def main():
    # Initialize Pygame
    pygame.init()

    # Set up the screen
    screen_width, screen_height = 1000, 650  # Increase the screen height to accommodate the item buttons
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Grid with Buildings, Robot, Police, Cars")

    # Load building, robot, police, car1, and car2 images
    # (Assuming these resources are loaded properly)
    building_image = pygame.image.load("building_logo.jpg")
    robot_image = pygame.image.load("robot.jpg")
    police_image = pygame.image.load("police.jpg")
    car1_image = pygame.image.load("car1.jpg")
    car2_image = pygame.image.load("car2.jpg")

    # Load item images
    item1_image = pygame.image.load("item1.jpeg")
    item2_image = pygame.image.load("item2.jpeg")
    item3_image = pygame.image.load("item3.jpeg")
    item4_image = pygame.image.load("item4.png")
    item5_image = pygame.image.load("item5.png")
    item6_image = pygame.image.load("item6.jpeg")
    item7_image = pygame.image.load("item7.jpg")
    item8_image = pygame.image.load("item8.jpeg")

    # Create a list of item images
    item_images = [item1_image, item2_image, item3_image, item4_image, item5_image, item6_image, item7_image, item8_image]

    # Define grid size
    rows, cols = 15, 15

    # Generate random building locations
    building_locations = []
    for _ in range(10):
        building_locations.append(generate_location(rows, cols, building_locations))

    # Generate random locations for robot, police, car1, and car2, ensuring they don't overlap with buildings
    robot_location = (0, 0)
    police_location = generate_location(rows, cols, building_locations + [robot_location])
    car1_image_locations = []
    for _ in range(2):
        car1_image_locations.append(generate_location(rows, cols, building_locations + [robot_location] + [police_location] + car1_image_locations))
    car2_image_locations = []
    for _ in range(2):
        car2_image_locations.append(generate_location(rows, cols, building_locations + [robot_location] + [police_location] + car1_image_locations + car2_image_locations))

    prev_direction_police = None  # Initialize previous direction for police as None
    prev_direction_car1 = [None, None]  # Initialize previous direction for car1s as None for both cars
    prev_direction_car2 = [None, None]  # Initialize previous direction for car2s as None for both cars

    # Calculate cell width and height
    cell_width = screen_width // cols
    cell_height = (screen_height - 50) // rows  # Subtract 50 for the height of the item buttons row

    item_button_size = 50

    # Calculate the total width needed for the item buttons
    total_item_width = item_button_size * len(item_images)

    # Calculate the starting position to center the item buttons horizontally
    start_x = 300
    button_rects = []
    for i, item_image in enumerate(item_images):
        button_rect = pygame.Rect(start_x + i * item_button_size, screen_height - item_button_size, item_button_size,
                                   item_button_size)
        button_rects.append(button_rect)
    item_name = ["item0", "item1", "item2", "item3", "item4", "item5", "item6", "item7"]
    value = 100

    # List to store clicked delivery points
    clicked_points = []

    del_x = None
    del_y = None
    click_on_delivery = None
    goal_location = None  # Initialize goal_location

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pass
            
                # Check if the mouse click was on an item button
            # Check if the mouse click was on an item button
                mouse_x, mouse_y = event.pos
                for i, button_rect in enumerate(delivery_buttons_rects):
                    if button_rect.collidepoint(mouse_x, mouse_y):
                        if click_on_delivery is None:
                            click_on_delivery = 1
                            # Get the delivery point coordinates corresponding to the clicked button
                            del_x, del_y = delivery_points[i]
                            print("Clicked Delivery Point Location:", del_x, del_y)
                            # Calculate the screen coordinates for displaying the clicked point
                            x = del_x * cell_width + cell_width // 2
                            y = del_y * cell_height + cell_height // 2
                            display_coordinates(x, y, screen, cell_width, cell_height, point_image_resized, delivery_points)
                            clicked_points.append((x, y))  # Add the clicked point to the list


                if del_x is not None and del_y is not None:
                    goal_location = (del_x, del_y)
                    # Clear the delivery point variables for the next delivery
                    del_x, del_y = None, None

                

                if mouse_x >= 300 and mouse_x < 350 and mouse_y >= 600 and mouse_y < 650:
                    value = 0
                elif mouse_x >= 350 and mouse_x < 400 and mouse_y >= 600 and mouse_y < 650:
                    value = 1
                elif mouse_x >= 400 and mouse_x < 450 and mouse_y >= 600 and mouse_y < 650:
                    value = 2
                elif mouse_x >= 450 and mouse_x < 500 and mouse_y >= 600 and mouse_y < 650:
                    value = 3
                elif mouse_x >= 500 and mouse_x < 550 and mouse_y >= 600 and mouse_y < 650:
                    value = 4
                elif mouse_x >= 550 and mouse_x < 600 and mouse_y >= 600 and mouse_y < 650:
                    value = 5
                elif mouse_x >= 600 and mouse_x < 650 and mouse_y >= 600 and mouse_y < 650:
                    value = 6
                elif mouse_x >= 650 and mouse_x < 700 and mouse_y >= 600 and mouse_y < 650:
                    value = 7

                if len(item_name) > value:
                    handle_item_click(value, item_images, screen, rows, cols)
                    del item_name[value]
                    value = 100

        if robot_location == goal_location:
            click_on_delivery = None
            

        # Move the police
        police_location, prev_direction_police = move_police(police_location, rows, cols, building_locations,
                                                              robot_location, car1_image_locations,
                                                              car2_image_locations, prev_direction_police)

        # Move car1s
        for i, car1_location in enumerate(car1_image_locations):
            car1_image_locations[i], prev_direction_car1[i] = move_car1(car1_location, rows, cols,
                                                                        building_locations, robot_location,
                                                                        police_location, car2_image_locations,
                                                                        prev_direction_car1[i])

        # Move car2s
        for i, car2_location in enumerate(car2_image_locations):
            car2_image_locations[i], prev_direction_car2[i] = move_car2(car2_location, rows, cols,
                                                                        building_locations, robot_location,
                                                                        police_location, car1_image_locations,
                                                                        prev_direction_car2[i])

        # Draw grid with entities
        draw_grid_with_entities(screen, screen_width, screen_height - 50, rows, cols, building_image,
                                building_locations, robot_image, robot_location, police_image, police_location,
                                car1_image, car1_image_locations, car2_image, car2_image_locations)

        # Generate random delivery points
        delivery_points = generate_delivery_points(rows, cols, building_locations + [robot_location] + [police_location] + car1_image_locations + car2_image_locations)

        # Create buttons for delivery points
        delivery_buttons_rects = delivery_buttons(delivery_points, cell_width, cell_height)

        # Resize the delivery point image based on cell size
        point_image = pygame.image.load("point.jpeg")
        point_image_width = cell_width  # Adjust the width to half of the cell width
        point_image_height = cell_height  # Adjust the height to half of the cell height
        point_image_resized = pygame.transform.scale(point_image, (point_image_width, point_image_height))

        # Draw delivery points
        for delivery_point in delivery_points:
            delivery_x = delivery_point[1] * cell_width + (cell_width - point_image_width) // 2  # Adjust X position to center the point image
            delivery_y = delivery_point[0] * cell_height + (cell_height - point_image_height) // 2  # Adjust Y position to center the point image

            # Check if the delivery point is currently clicked
            if (delivery_point[1] * cell_width + cell_width // 2,
                delivery_point[0] * cell_height + cell_height // 2) in clicked_points:
                # Create a red transparent surface
                red_surface = pygame.Surface((point_image_width, point_image_height), pygame.SRCALPHA)
                red_surface.fill((255, 0, 0, 128))  # Red color with 50% transparency

                # Blit the red transparent surface onto the delivery point image
                screen.blit(point_image_resized, (delivery_x, delivery_y))
                screen.blit(red_surface, (delivery_x, delivery_y))
            else:
                # Draw the delivery point image
                screen.blit(point_image_resized, (delivery_x, delivery_y))

        if goal_location is not None:
            optimal_path = best_first_search(robot_location, goal_location, building_locations)

            if optimal_path:
                # Move the robot along the optimal path
                if len(optimal_path) > 1:  # Check if there are steps to move
                    next_location = optimal_path[1]  # Get the next location to move
                    if is_blocked_or_occupied(next_location, building_locations, car1_image_locations, car2_image_locations , police_location):
                        print("Next cell is occupied by a car. Waiting for", CAR_CHECK_DELAY, "seconds...")
                        time.sleep(CAR_CHECK_DELAY)  
                        if not is_blocked_or_occupied(next_location, building_locations, car1_image_locations, car2_image_locations , police_location):
                             # If the cell is now free, move the robot
                             robot_location = next_location
                             print("Robot Location:", robot_location)
                    else:
                        robot_location = next_location  # Update the robot's location
                        print("Robot Location:", robot_location)  # Print the location of the robot after every move
        # Check if the robot reaches the goal and remove the delivery point
        if robot_location == goal_location:
            for i, point in enumerate(delivery_points):
                if point == goal_location:
                    del delivery_points[i]  # Remove the delivery point from the list
                    break  # Exit the loop after removing the point

        # Draw item buttons
        draw_item_buttons(screen, item_images)

        pygame.display.flip()
        pygame.time.delay(CAR_MOVEMENT_DELAY)

    pygame.quit()

if __name__ == "__main__":
    main()


Clicked Delivery Point Location: 1 8
Clicked on delivery point at (x=99, y=340)
Robot Location: (0, 1)
Robot Location: (0, 2)
Robot Location: (0, 3)
Robot Location: (0, 4)
Robot Location: (0, 5)
Robot Location: (0, 6)
Robot Location: (0, 7)
Robot Location: (0, 8)
Robot Location: (1, 8)
Clicked Delivery Point Location: 6 7
Clicked on delivery point at (x=429, y=300)
Robot Location: (2, 8)
Robot Location: (3, 8)
Robot Location: (4, 8)
Robot Location: (5, 8)
Robot Location: (5, 7)
Robot Location: (6, 7)
Clicked Delivery Point Location: 10 10
Clicked on delivery point at (x=693, y=420)
Robot Location: (7, 7)
Robot Location: (7, 8)
Robot Location: (8, 8)
Robot Location: (8, 9)
Robot Location: (9, 9)
Robot Location: (9, 10)
Robot Location: (10, 10)
Clicked Delivery Point Location: 5 14
Clicked on delivery point at (x=363, y=580)
Robot Location: (9, 10)
Robot Location: (8, 10)
Robot Location: (8, 11)
Robot Location: (7, 11)
Robot Location: (7, 12)
Robot Location: (6, 12)
Robot Location: (6, 1